In [1]:
import torch
import torch.nn.functional as F
import torch.optim as optim

from torchpack.utils.config import configs

from torchquantum.datasets import MNIST
from torch.optim.lr_scheduler import CosineAnnealingLR


import random
import numpy as np

In [2]:
def train(dataflow, model, device, optimizer):
    for feed_dict in dataflow['train']:
        inputs = feed_dict['image'].to(device)
        targets = feed_dict['digit'].to(device)

        outputs = model(inputs)
        loss = F.nll_loss(outputs, targets)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        print(f"loss: {loss.item()}", end='\r')

In [3]:
def valid_test(dataflow, split, model, device, qiskit=False):
    target_all = []
    output_all = []
    with torch.no_grad():
        for feed_dict in dataflow[split]:
            inputs = feed_dict['image'].to(device)
            targets = feed_dict['digit'].to(device)

            outputs = model(inputs)

            target_all.append(targets)
            output_all.append(outputs)
        target_all = torch.cat(target_all, dim=0)
        output_all = torch.cat(output_all, dim=0)

    _, indices = output_all.topk(1, dim=1)
    masks = indices.eq(target_all.view(-1, 1).expand_as(indices))
    size = target_all.shape[0]
    corrects = masks.sum().item()
    accuracy = corrects / size
    loss = F.nll_loss(output_all, target_all).item()

    print(f"{split} set accuracy: {accuracy}")
    print(f"{split} set loss: {loss}")

In [4]:
from q10digit_models import Q10DigitFCModel0

In [5]:
def main(epochs=5):

    seed = 0
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)

    dataset = MNIST(
        root='./mnist_data',
        train_valid_split_ratio=[0.9, 0.1],
        #digits_of_interest=[3, 6],
        n_test_samples=100,
        n_train_samples=500,
    )
    dataflow = dict()

    for split in dataset:
        sampler = torch.utils.data.RandomSampler(dataset[split])
        dataflow[split] = torch.utils.data.DataLoader(
            dataset[split],
            batch_size=10,
            sampler=sampler,
            num_workers=8,
            pin_memory=True)

    use_cuda = torch.cuda.is_available()
    device = torch.device("cuda" if use_cuda else "cpu")

    configs.load("/home/fangw/quantum-dp/pytorch-quantum-master/examples/configs/mnist/ten/default.yml", recursive=True)
    configs.model.arch.n_blocks = 8

    model = Q10DigitFCModel0(arch=configs.model.arch).to(device)

    n_epochs = epochs
    optimizer = optim.Adam(model.parameters(), lr=5e-3, weight_decay=1e-4)
    scheduler = CosineAnnealingLR(optimizer, T_max=n_epochs)

    for epoch in range(1, n_epochs + 1):
        # train
        print(f"Epoch {epoch}:")
        train(dataflow, model, device, optimizer)
        print(optimizer.param_groups[0]['lr'])

        # valid
        valid_test(dataflow, 'valid', model, device)
        scheduler.step()

    # test
    valid_test(dataflow, 'test', model, device, qiskit=False)

    return model


In [6]:
model  = main(15)

[2023-05-02 19:05:29.954] Only use the front 500 images as TRAIN set.
[2023-05-02 19:05:30.069] Only use the front 100 images as TEST set.


Epoch 1:
0.005 2.1975343227386475
valid set accuracy: 0.2735
valid set loss: 2.228492021560669
Epoch 2:
0.0049453690018345143066
valid set accuracy: 0.2996666666666667
valid set loss: 2.193239688873291
Epoch 3:
0.0047838636441065028066
valid set accuracy: 0.322
valid set loss: 2.168720245361328
Epoch 4:
0.0045225424859373685205
valid set accuracy: 0.3438333333333333
valid set loss: 2.150378704071045
Epoch 5:
0.0041728265158971455077
valid set accuracy: 0.3506666666666667
valid set loss: 2.140296697616577
Epoch 6:
0.00375.9743868112564087
valid set accuracy: 0.3601666666666667
valid set loss: 2.128539800643921
Epoch 7:
0.0032725424859373687326
valid set accuracy: 0.358
valid set loss: 2.121077537536621
Epoch 8:
0.0027613211581691345925
valid set accuracy: 0.36283333333333334
valid set loss: 2.1155803203582764
Epoch 9:
0.0022386788418308675625
valid set accuracy: 0.3635
valid set loss: 2.110771417617798
Epoch 10:
0.0017274575140626328125
valid set accuracy: 0.36516666666666664
valid set 

In [7]:
from qdp import evaluate
import numpy as np
from torchquantum.plugins import tq2qiskit
from qiskit2cirq import qiskit2cirq

q_layer_circ = tq2qiskit(model.q_device, model.q_layer)
circuit = qiskit2cirq(q_layer_circ)

In [8]:
evaluate(circuit, [np.array([[0,0],[0,1]]), np.array([[1,0],[0,0]])])

depolarize, 0.01
18.902539014816284
depolarize, 0.001
17.438279628753662
depolarize, 0.0001
17.828185081481934
depolarize, 1e-05
16.818902492523193
depolarize, 1e-06
17.123612642288208
bit_flip, 0.01
17.39156174659729
bit_flip, 0.001
17.13722801208496
bit_flip, 0.0001
17.5580894947052
bit_flip, 1e-05
16.818037748336792
bit_flip, 1e-06
17.118456602096558
+------------+---------+----------+-------+
& noise type & noisy p & kappa    & time  &
+------------+---------+----------+-------+
& depolarize & 0.01    & 1.170    & 18.90 &
& depolarize & 0.001   & 7.241    & 17.44 &
& depolarize & 0.0001  & 71.396   & 17.83 &
& depolarize & 1e-05   & 712.899  & 16.82 &
& depolarize & 1e-06   & 7045.995 & 17.12 &
& bit_flip   & 0.01    & 1.132    & 17.39 &
& bit_flip   & 0.001   & 6.677    & 17.14 &
& bit_flip   & 0.0001  & 66.050   & 17.56 &
& bit_flip   & 1e-05   & 660.314  & 16.82 &
& bit_flip   & 1e-06   & 6663.058 & 17.12 &
+------------+---------+----------+-------+
